In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 23 08:39:11 2021

@author: lpott
"""
import argparse
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm

from preprocessing import *
from dataset import *
from metrics import *
from model import *
from utils import bert2dict


In [3]:
# variables

read_filename ="ml-1m\\ratings.dat"
read_bert_filename = "bert_sequence_20m.txt"
read_movie_filename = ""#"movies-1m.csv"
size = "1m"

num_epochs = 100
lr = 1e-3
batch_size = 64
reg = 1e-5
train_method = "normal"


hidden_dim = 0
embedding_dim = 64
bert_dim= 0
window = 0

freeze_plot = False
tied = False
dropout= 0

k = 10
max_length = 200
min_len = 10


# nextitnet options...
hidden_layers = 3
dilations = [1,2,2,4]

model_type = "nextitnet"

In [4]:
torch.cuda.empty_cache()

In [5]:
# ------------------Data Initialization----------------------#

# convert .dat file to time-sorted pandas dataframe
ml_1m = create_df(read_filename,size=size)

# remove users who have sessions lengths less than min_len
ml_1m = filter_df(ml_1m,item_min=min_len)

========== Creating DataFrame ==========


C:\Users\lpott\Desktop\UCLA\COMSCI247-80\attentive-session-based-recs\preprocessing.py:32: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(filename,sep='::',header=None)


user_id        6040
item_id        3706
rating            5
timestamp    458455
dtype: int64
(1000209, 4)
Minimum Session Length: 20
Maximum Session Length: 2314
Average Session Length: 165.60
========== Filtering Sessions <= 10  DataFrame ==========
user_id        6040
item_id        3706
rating            5
timestamp    458455
dtype: int64
(1000209, 4)
Minimum Session Length: 20
Maximum Session Length: 2314
Average Session Length: 165.60


In [6]:
# ------------------Data Initialization----------------------#
if read_movie_filename != "":
    ml_movie_df = create_movie_df(read_movie_filename,size=size)
    ml_movie_df = convert_genres(ml_movie_df)
    
    # initialize reset object
    reset_object = reset_df()
    
    # map all user ids, item ids, and genres to range 0 - number of users/items/genres
    ml_1m,ml_movie_df = reset_object.fit_transform(ml_1m,ml_movie_df)
    
    # value that padded genre tokens shall take
    pad_genre_token = reset_object.genre_enc.transform(["NULL"]).item()
    
    genre_dim = len(np.unique(np.concatenate(ml_movie_df.genre))) - 1

else:
    # initialize reset object
    reset_object = reset_df()
    
    # map all user ids and item ids to range 0 - Number of Users/Items 
    # i.e. [1,7,5] -> [0,2,1]
    ml_1m = reset_object.fit_transform(ml_1m)
    
    pad_genre_token = None
    ml_movie_df = None
    genre_dim = 0

========== Initialize Reset DataFrame Object ==========
========== Resetting user ids and item ids in DataFrame ==========


In [7]:
# ------------------Data Initialization----------------------#
# how many unique users, items, ratings and timestamps are there
n_users,n_items,n_ratings,n_timestamp = ml_1m.nunique()

# value that padded tokens shall take
pad_token = n_items

# the output dimension for softmax layer
output_dim = n_items


# get the item id : bert plot embedding dictionary
if bert_dim != 0:
    feature_embed = bert2dict(bert_filename=read_bert_filename)

In [8]:
# create a dictionary of every user's session (history)
# i.e. {user: [user clicks]}
if size == "1m":
    user_history = create_user_history(ml_1m)

elif size == "20m":
    import pickle
    with open('userhistory.pickle', 'rb') as handle:
        user_history = pickle.load(handle)
# create a dictionary of all items a user has not clicked
# i.e. {user: [items not clicked by user]}
# user_noclicks = create_user_noclick(user_history,ml_1m,n_items)

  1%|█                                                                              | 82/6040 [00:00<00:07, 818.89it/s]

========== Creating User Histories ==========


100%|█████████████████████████████████████████████████████████████████████████████| 6040/6040 [00:06<00:00, 920.17it/s]


In [9]:
#import pickle

#with open('userhistory.pickle', 'wb') as handle:
#    pickle.dump(user_history, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('userhistory.pickle', 'rb') as handle:
#    user_history = pickle.load(handle)

In [10]:
# split data by leave-one-out strategy
# have train dictionary {user: [last 41 items prior to last 2 items in user session]}
# have val dictionary {user: [last 41 items prior to last item in user session]}
# have test dictionary {user: [last 41 items]}
# i.e. if max_length = 4, [1,2,3,4,5,6] -> [1,2,3,4] , [2,3,4,5] , [3,4,5,6]
train_history,val_history,test_history = train_val_test_split(user_history,max_length=max_length)

# initialize the train,validation, and test pytorch dataset objects
# eval pads all items except last token to predict
train_dataset = GRUDataset(train_history,genre_df=ml_movie_df,mode='train',max_length=max_length,pad_token=pad_token,pad_genre_token=pad_genre_token)
val_dataset = GRUDataset(val_history,genre_df=ml_movie_df,mode='eval',max_length=max_length,pad_token=pad_token,pad_genre_token=pad_genre_token)
test_dataset = GRUDataset(test_history,genre_df=ml_movie_df,mode='eval',max_length=max_length,pad_token=pad_token,pad_genre_token=pad_genre_token)

# create the train,validation, and test pytorch dataloader objects
train_dl = DataLoader(train_dataset,batch_size = batch_size,shuffle=True)
val_dl = DataLoader(val_dataset,batch_size=64)
test_dl = DataLoader(test_dataset,batch_size=64)

100%|██████████████████████████████████████████████████████████████████████████| 6040/6040 [00:00<00:00, 208828.37it/s]

========== Splitting User Histories into Train, Validation, and Test Splits ==========


In [11]:
print("Bert dim: {:d}".format(bert_dim))
print("Genre dim: {:d}".format(genre_dim))
print("Pad Token: {}".format(pad_token))
print("Pad Genre Token: {}".format(pad_genre_token))

Bert dim: 0
Genre dim: 0
Pad Token: 3706
Pad Genre Token: None


In [12]:
# ------------------Model Initialization----------------------#

# initialize gru4rec model with arguments specified earlier
if model_type == "feature_add":
    model = gru4recF(embedding_dim=embedding_dim,
             hidden_dim=hidden_dim,
             output_dim=output_dim,
             genre_dim=genre_dim,
             batch_first=True,
             max_length=max_length,
             pad_token=pad_token,
             pad_genre_token=pad_genre_token,
             bert_dim=bert_dim,
             tied = tied,
             dropout=dropout)


if model_type == "feature_concat":
    model = gru4recFC(embedding_dim=embedding_dim,
             hidden_dim=hidden_dim,
             output_dim=output_dim,
             genre_dim=genre_dim,
             batch_first=True,
             max_length=max_length,
             pad_token=pad_token,
             pad_genre_token=pad_genre_token,
             bert_dim=bert_dim,
             tied = tied,
             dropout=dropout)

if model_type == "vanilla":
    model = gru4rec_vanilla(hidden_dim=hidden_dim,
                            output_dim=output_dim,
                            batch_first=True,
                            max_length=max_length,
                            pad_token=pad_token,
                            tied=tied,
                            embedding_dim=embedding_dim)

if model_type =="feature_only":
    model = gru4rec_feature(hidden_dim=hidden_dim,
                            output_dim=output_dim,
                            batch_first=True,
                            max_length=max_length,
                            pad_token=pad_token,
                            bert_dim=bert_dim)

if model_type == "conv":
    model = gru4rec_conv(embedding_dim,
                 hidden_dim,
                 output_dim,
                 batch_first=True,
                 max_length=200,
                 pad_token=0,
                 dropout=0,
                 window=3,
                 tied=tied)
    
if model_type == "nextitnet":
    model = NextItNet(embedding_dim=embedding_dim,
                      output_dim=output_dim,
                      hidden_layers=hidden_layers,
                      dilations=dilations,
                      pad_token=n_items,
                      max_len=max_length)

In [13]:
if bert_dim != 0:
    model.init_weight(reset_object,feature_embed)
    
model = model.cuda()

In [14]:
[name for name,param in model.named_parameters() if (("movie" not in name) or ("plot_embedding" in name) or ("genre" in name))]

['item_embedding.weight',
 'hidden_layers.0.0.reduce.weight',
 'hidden_layers.0.0.reduce.bias',
 'hidden_layers.0.0.masked.weight',
 'hidden_layers.0.0.masked.bias',
 'hidden_layers.0.0.increase.weight',
 'hidden_layers.0.0.increase.bias',
 'hidden_layers.0.0.reduce_norm.weight',
 'hidden_layers.0.0.reduce_norm.bias',
 'hidden_layers.0.0.masked_norm.weight',
 'hidden_layers.0.0.masked_norm.bias',
 'hidden_layers.0.0.increase_norm.weight',
 'hidden_layers.0.0.increase_norm.bias',
 'hidden_layers.0.1.reduce.weight',
 'hidden_layers.0.1.reduce.bias',
 'hidden_layers.0.1.masked.weight',
 'hidden_layers.0.1.masked.bias',
 'hidden_layers.0.1.increase.weight',
 'hidden_layers.0.1.increase.bias',
 'hidden_layers.0.1.reduce_norm.weight',
 'hidden_layers.0.1.reduce_norm.bias',
 'hidden_layers.0.1.masked_norm.weight',
 'hidden_layers.0.1.masked_norm.bias',
 'hidden_layers.0.1.increase_norm.weight',
 'hidden_layers.0.1.increase_norm.bias',
 'hidden_layers.0.2.reduce.weight',
 'hidden_layers.0.2.re

In [15]:
[name for name,param in model.named_parameters() if ("plot" not in name) and ("genre" not in name)]

['item_embedding.weight',
 'hidden_layers.0.0.reduce.weight',
 'hidden_layers.0.0.reduce.bias',
 'hidden_layers.0.0.masked.weight',
 'hidden_layers.0.0.masked.bias',
 'hidden_layers.0.0.increase.weight',
 'hidden_layers.0.0.increase.bias',
 'hidden_layers.0.0.reduce_norm.weight',
 'hidden_layers.0.0.reduce_norm.bias',
 'hidden_layers.0.0.masked_norm.weight',
 'hidden_layers.0.0.masked_norm.bias',
 'hidden_layers.0.0.increase_norm.weight',
 'hidden_layers.0.0.increase_norm.bias',
 'hidden_layers.0.1.reduce.weight',
 'hidden_layers.0.1.reduce.bias',
 'hidden_layers.0.1.masked.weight',
 'hidden_layers.0.1.masked.bias',
 'hidden_layers.0.1.increase.weight',
 'hidden_layers.0.1.increase.bias',
 'hidden_layers.0.1.reduce_norm.weight',
 'hidden_layers.0.1.reduce_norm.bias',
 'hidden_layers.0.1.masked_norm.weight',
 'hidden_layers.0.1.masked_norm.bias',
 'hidden_layers.0.1.increase_norm.weight',
 'hidden_layers.0.1.increase_norm.bias',
 'hidden_layers.0.2.reduce.weight',
 'hidden_layers.0.2.re

In [16]:
# initialize Adam optimizer with gru4rec model parameters
if train_method != "normal":
    optimizer_features = torch.optim.Adam([param for name,param in model.named_parameters() if (("movie" not in name) or ("plot_embedding" in name) or ("genre" in name)) ],
                                          lr=lr/10,weight_decay=reg)
    
    optimizer_ids = torch.optim.Adam([param for name,param in model.named_parameters() if ("plot" not in name) and ("genre" not in name)],
                                     lr=lr,weight_decay=reg)

elif train_method == "normal":
    optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=reg)
    
if freeze_plot and bert_dim !=0:
    model.plot_embedding.weight.requires_grad = False

In [17]:
loss_fn = nn.CrossEntropyLoss(ignore_index=n_items)
#Recall_Object = Recall_E_prob(ml_1m,user_history,n_users,n_items,k=k)
#Recall_Object = Recall_E_Noprob(ml_1m,user_history,n_users,n_items,k=k)

In [18]:
Recall_Object = Recall_E_prob(ml_1m,user_history,n_users,n_items,k=k)

========== Creating Hit@10 Metric Object ==========


In [19]:
#print("Baseline POP results: ",Recall_Object.popular_baseline())

In [20]:
#training_hit = Recall_Object(model,train_dl)
#validation_hit = Recall_Object(model,val_dl)
#testing_hit = Recall_Object(model,test_dl)
#print("Training Hits@{:d}: {:.2f}".format(k,training_hit))
#print("Validation Hits@{:d}: {:.2f}".format(k,validation_hit))
#print("Testing Hits@{:d}: {:.2f}".format(k,testing_hit))

In [21]:
model_type

'nextitnet'

In [ ]:
# ------------------Training Initialization----------------------#
max_train_hit = 0
max_val_hit = 0
max_test_hit = 0

for epoch in range(num_epochs):
    print("="*20,"Epoch {}".format(epoch+1),"="*20)
    
    model.train()  
    
    running_loss = 0

    for j,data in enumerate(tqdm(train_dl,position=0,leave=True)):
        
        if train_method != "normal":
            optimizer_features.zero_grad()
            optimizer_ids.zero_grad()
            
        elif train_method == "normal": 
            optimizer.zero_grad()
        
        if genre_dim != 0:            
            inputs,genre_inputs,labels,x_lens,uid = data
            outputs = model(x=inputs.cuda(),x_lens=x_lens.squeeze().tolist(),x_genre=genre_inputs.cuda())
        
        elif genre_dim == 0:
            inputs,labels,x_lens,uid = data 
            outputs = model(x=inputs.cuda(),x_lens=x_lens.squeeze().tolist())
       
        if tied:
            outputs_ignore_pad = outputs[:,:,:-1]
            loss = loss_fn(outputs_ignore_pad.view(-1,outputs_ignore_pad.size(-1)),labels.view(-1).cuda())
            
        else:
            loss = loss_fn(outputs.view(-1,outputs.size(-1)),labels.view(-1).cuda())
            
        loss.backward()
        
        if train_method != "normal":
            if train_method == "interleave":
                # interleave on the epochs
                if (j+1) % 2 == 0:
                    optimizer_features.step()
                else:
                    optimizer_ids.step()

            elif train_method == "alternate":
                if (epoch+1) % 2 == 0:
                    optimizer_features.step()
                else:
                    optimizer_ids.step()

        elif train_method == "normal":
            optimizer.step()

        running_loss += loss.detach().cpu().item()

    del outputs
    torch.cuda.empty_cache()
    training_hit = Recall_Object(model,train_dl,"train")
    validation_hit = Recall_Object(model,val_dl,"validation")
    testing_hit = Recall_Object(model,test_dl,"test")
    
    if max_val_hit < validation_hit:
        max_val_hit = validation_hit
        max_test_hit = testing_hit
        max_train_hit = training_hit
    
    torch.cuda.empty_cache()
    print("Training CE Loss: {:.5f}".format(running_loss/len(train_dl)))
    print("Training Hits@{:d}: {:.2f}".format(k,training_hit))
    print("Validation Hits@{:d}: {:.2f}".format(k,validation_hit))
    print("Testing Hits@{:d}: {:.2f}".format(k,testing_hit))


print("="*100)
print("Maximum Training Hit@{:d}: {:.2f}".format(k,max_train_hit))
print("Maximum Validation Hit@{:d}: {:.2f}".format(k,max_val_hit))
print("Maximum Testing Hit@{:d}: {:.2f}".format(k,max_test_hit))

  0%|                                                                                           | 0/95 [00:00<?, ?it/s]

==================== Epoch 1 ====================



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\lpott\anaconda3\envs\NLP\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-272a4c62a179>", line 28, in <module>
    outputs = model(x=inputs.cuda(),x_lens=x_lens.squeeze().tolist())
  File "C:\Users\lpott\anaconda3\envs\NLP\lib\site-packages\torch\nn\modules\module.py", line 547, in __call__
    result = self.forward(*input, **kwargs)
TypeError: forward() got an unexpected keyword argument 'x_lens'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\lpott\anaconda3\envs\NLP\lib\site-packages\IPython\core\interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last

In [22]:
print("="*100)
print("Maximum Training Hit@{:d}: {:.2f}".format(k,max_train_hit))
print("Maximum Validation Hit@{:d}: {:.2f}".format(k,max_val_hit))
print("Maximum Testing Hit@{:d}: {:.2f}".format(k,max_test_hit))

Maximum Training Hit@10: 0.00
Maximum Validation Hit@10: 0.00
Maximum Testing Hit@10: 0.00
ERROR! Session/line number was not unique in database. History logging moved to new session 4224
